# Свертка и предвыборка

1. Построить нейросети с линейными, сверточными слоями и слоями подвыборки.
2. Проверить, какая конфигурация работает лучше линейных слоев.
3. Оценить качество предсказания по коэффициенту сходства.

## Подключение библиотек.

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras

from sklearn.model_selection import train_test_split
from skimage import io
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D

## Используемые функции

In [3]:
image_x = 525
image_y = 350

def mask_rate(a, x, y):
    b = a // 1400
    return np.round(x*(b*x // 2100) + y*(a%1400) // 1400).astype("uint32")

def calc_mask(px, x=525, y=350):
    p = np.array([int(n) for n in px.split(" ")]).reshape(-1, 2)
    mask = np.zeros(y*x, dtype="uint8")
    for i, m in p:
        mask[mask_rate(i, x, y)-1:mask_rate(m + i, x, y)] = 1
    return mask.reshape(y, x).transpose()

def calc_dice(x):
    dice = 0
    px = x["EncodedPixels"]
    if px != px and x["target"] == 0:
        dice = 1
    elif px == px and x["target"] == 1:
        mask = calc_mask(px).flatten()
        target = np.ones(image_x*image_y, dtype="uint8")
        dice = 2*np.sum(target[mask==1]) / (np.sum(target) + np.sum(mask))
    
    return dice

## Загрузка данных

In [4]:
data = pd.read_csv("../data/clouds/train.csv.gz")

In [5]:
data["Image"] = data["Image_Label"].str.split("_").str[0]
data["Label"] = data["Image_Label"].str.split("_").str[1]
data.drop(labels=["Image_Label"], axis=1, inplace=True)
data_fish = data[data["Label"] == "Fish"]
print(data_fish.head())

                                        EncodedPixels        Image Label
0   264918 937 266318 937 267718 937 269118 937 27...  0011165.jpg  Fish
4   233813 878 235213 878 236613 878 238010 881 23...  002be4f.jpg  Fish
8   3510 690 4910 690 6310 690 7710 690 9110 690 1...  0031ae9.jpg  Fish
12                                                NaN  0035239.jpg  Fish
16  2367966 18 2367985 2 2367993 8 2368002 62 2369...  003994e.jpg  Fish


## Разделение данных

In [6]:
train, test = train_test_split(data_fish, test_size=0.2)
del data
print(train.head())

      EncodedPixels        Image Label
11972           NaN  888f3b3.jpg  Fish
2532            NaN  1c3a47b.jpg  Fish
19816           NaN  e492617.jpg  Fish
21912           NaN  fcf5591.jpg  Fish
19664           NaN  e317214.jpg  Fish
